# Best Practices

This document collects some of the best practices used elsewhere in the pandas documentation.
Together, they lead to a style of code lovingly referred to as *pandorable*. We encourage
you to apply these practicies when using pandas.

In [1]:
import pandas as pd
pd.options.display.max_rows = 10

## Use method chaining

Compare the following two stories (credit to [Jeff Allen](http://trestletech.com/wp-content/uploads/2015/07/dplyr.pdf))

First,

```python
on_hill = went_up(jack_jill, 'hill')
with_water = fetch(on_hill, 'water')
fallen = fell_down(with_water, 'jack')
broken = broke(fallen, 'jack')
after = tmple_after(broken, 'jill')
```

and second,

```python
(jack_jill
    .went_up("hill")
    .fetch("water")
    .fell_down("jack")
    .broke("crown")
    .tumble_after("jill"))
```

I hope you agree that the second story, written in a method chaining style, is easier to follow. It avoids uninteresting intermediate variables, generally making things easier to read.

As a concrete example, we'll look at the light pre-procesing done to the `airports` datset following Hadley Wickham's [nycflights13 package](https://github.com/hadley/nycflights13/blob/master/data-raw/airports.R).

In [2]:
names = ["id", "name", "city", "country", "faa", "icao", "lat", "lon", "alt", "tz", "dst", "tzone"]

airports_raw = pd.read_csv(
    "https://raw.githubusercontent.com/hadley/nycflights13/master/data-raw/airports.dat",
    header=None, names=names)
airports_raw.head()

,id,name,city,country,faa,icao,lat,lon,alt,tz,dst,tzone
0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10.0,U,Pacific/Port_Moresby
1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.788700,20,10.0,U,Pacific/Port_Moresby
2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10.0,U,Pacific/Port_Moresby
3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10.0,U,Pacific/Port_Moresby
4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.220050,146,10.0,U,Pacific/Port_Moresby


We'll do a bit of cleaning up including filtering the rows and columns to the values of interest.

In [3]:
airports = (
    airports_raw
        .loc[lambda df: (df['country'] == 'United States') & (df['faa'] != '')]
        [['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst', 'tzone']]
        .drop_duplicates(subset="faa")
        .set_index("faa")
)
airports

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
4I7,Putnam County Airport,39.633556,-86.813806,842,-5.0,U,America/New_York
C91,Dowagiac Municipal Airport,41.992934,-86.128012,748,-5.0,U,America/New_York
CDI,Cambridge Municipal Airport,39.975028,-81.577583,799,-5.0,U,America/New_York
SUE,Door County Cherryland Airport,44.843667,-87.421556,725,-6.0,U,America/Chicago
0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5.0,U,America/New_York
...,...,...,...,...,...,...,...
UCA,Union Station,43.104167,-75.223333,456,-5.0,A,America/New_York
CVO,Corvallis Muni,44.506700,-123.291500,250,-8.0,A,America/Los_Angeles
CWT,Chatsworth Station,34.256944,-118.598889,978,-8.0,A,America/Los_Angeles


Most Series or DataFrame methods return a new Series or DataFrame, encouraging this method chaining style. Some notable methods include

1. :meth:`DataFrame.assign`
2. :meth:`DataFrame.loc`, :meth:`DataFrame.iloc`, :meth:`DataFrame.where`, and ``DataFrame.__getitem__`.
3. :meth:`DataFrame.pipe`

One thing to note, the `assign` and indexing methods will accept callables, which you use to refer to the previous link in the method chain. Consider translating an imperative string of operations like

```python
df1 = pd.read_csv(...)
df1['foo'] = df1['foo'].str.upper()
df1 = df1.loc[df['bar'] > 3]
```

to method chaining style. You'd use callables, often `lambda` functions, to refer to `df1` in subsequent operations.

```python
df = (
    pd.read_csv(...)
    .assign(foo=lambda df: df["foo"].str.upper())
    .loc[lambda df: df["bar"] > 3]
)
```

Finally, pandas provides an escape hatch through the `.pipe` method. With `.pipe`, you can provide any callable that expects a DataFrame (or Series) as it's first argument. For example, we could implement a function approximating the great circle distance between some airport `to` and the rest.

In [4]:
import numpy as np


def great_circle_distance(df, to="DSM"):
    # https://www.johndcook.com/blog/python_longitude_latitude/
    df = df.copy()
    lat = np.deg2rad(90 - df['lat'])
    lon = np.deg2rad(90 - df['lon'])
    
    to_lat, to_lon = df.loc[to, ['lat', 'lon']]
    cos = (np.sin(lat) * np.sin(to_lat) * np.cos(lon - to_lon) +
           np.cos(lat) * np.cos(to_lat))
           
    arc = np.arccos(cos)
    kilometers = 6373 * cos
    df[f'km_to_{to}'] = kilometers
    return df

In [5]:
great_circle_distance(airports)

,name,lat,lon,alt,tz,dst,tzone,km_to_DSM
faa,,,,,,,,
4I7,Putnam County Airport,39.633556,-86.813806,842,-5.0,U,America/New_York,-611.548347
C91,Dowagiac Municipal Airport,41.992934,-86.128012,748,-5.0,U,America/New_York,-874.022782
CDI,Cambridge Municipal Airport,39.975028,-81.577583,799,-5.0,U,America/New_York,-827.065264
SUE,Door County Cherryland Airport,44.843667,-87.421556,725,-6.0,U,America/Chicago,-1122.842947
0P2,Shoestring Aviation Airfield,39.794824,-76.647191,1000,-5.0,U,America/New_York,-997.427349
...,...,...,...,...,...,...,...,...
UCA,Union Station,43.104167,-75.223333,456,-5.0,A,America/New_York,-1370.623054
CVO,Corvallis Muni,44.506700,-123.291500,250,-8.0,A,America/Los_Angeles,-531.679746
CWT,Chatsworth Station,34.256944,-118.598889,978,-8.0,A,America/Los_Angeles,594.567494


Notice that our custom `great_circle_distance` function further encourages method chaining by returning a DataFrame itself.

Appending that to our original method chain, that would be

```python
airports = (
    airports_raw
        .loc[lambda df: (df['country'] == 'United States') & (df['faa'] != '')]
        [['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst', 'tzone']]
        .drop_duplicates(subset="faa")
        .set_index("faa")
        .pipe(gcd)
)
```

Additional keyword arguments passed to `.pipe` are passed through to the callable.

```python
airports = (
    ...
    .pipe(gcd, to="ORD")
)
```

## Use Meaningful Labels

Every Series and DataFrame has a `.index` property storing the *row labels*.
Additionally, DataFrame has the `.columns` property for storing *column labels*.

We recommend that you use meaningful labels. Pandas most fundamental operations are all based around the idea of *alignment by label*. Constructors, binary options (`add`, `mul`, etc.), reshaping (`concat`), etc. all align before doing an operation.

Let's consider a dummy example computing population density from two datasets (https://jakevdp.github.io/PythonDataScienceHandbook/03.03-operations-in-pandas.html).

In [6]:
area = pd.DataFrame([
    ('Alaska', 1723337),
    ('Texas', 695662),
    ('California', 423967)
], columns=['state', 'area'])
area

,state,area
0,Alaska,1723337
1,Texas,695662
2,California,423967


In [7]:
population = pd.DataFrame([
    ('California', 38332521),
    ('Texas', 26448193),
    ('New York', 19651127),
], columns=['state', 'population'])
population

,state,population
0,California,38332521
1,Texas,26448193
2,New York,19651127


If we naively divide the population column by the area column, we get incorrect results.

In [8]:
population['population'] / area['area']

0    22.243195
1    38.018740
2    46.350605
dtype: float64

It'd be better to model this data as two Series, each with the `state` as its index.

In [9]:
area_ = area.set_index("state")['area']
population_ = population.set_index("state")["population"]
population_ / area_

state
Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

Pandas uses row (and, for DataFrames, column labels) to align the data before doing the operation. This ensures that values and their labels propagate together through operations.

## Avoid duplicate row and column labels

One of pandas' primary roles is to help clean up messy tabular data. As such, it needs to support duplicates in the row labels. That does not mean, however, that you should allow duplicates to stick around; we recommend addressing duplicate labels as early as possible to avoid surpsises later on. Consider one of the most basic opertions: indexing. Duplicate labels can change the behavior in surprising ways.

Pandas follows the NumPy tradition of *reducing dimensionality* when indexing. Slicing a row from a 2-D array returns a 1-D array. Slicing a row and a column returns a scalar. Similarly with pandas.

In [10]:
airports['name']

faa
4I7                Putnam County Airport
C91           Dowagiac Municipal Airport
CDI          Cambridge Municipal Airport
SUE       Door County Cherryland Airport
0P2         Shoestring Aviation Airfield
                     ...                
UCA                        Union Station
CVO                       Corvallis Muni
CWT                   Chatsworth Station
DHB                 Deer Harbor Seaplane
OLT    San Diego Old Town Transit Center
Name: name, Length: 1459, dtype: object

In [11]:
airports.loc['BFT', 'name']

'Beaufort'

But, when there are duplicates in the index, it's no longer possible to reduce dimensionality.

In [12]:
airports_raw.set_index('faa').loc['BFT']

,id,name,city,country,icao,lat,lon,alt,tz,dst,tzone
faa,,,,,,,,,,,
BFT,3769,Beaufort,Beaufort,United States,KNBC,32.477411,-80.723161,37,-5.0,A,America/New_York
BFT,7049,BFT County Airport,Beauford,United States,KBFT,32.410830,-80.635000,500,-5.0,A,America/New_York


In this case, there are *two* rows with the code `FAA`, meaning the `.loc['BFT']` returns a DataFrame, rather than a Series.

## Avoid Inplace Operations

For many operations, Pandas current memory model doesn't allow true inplace (zero copy) operations.
The reasons are complicated, and we hope to address them someday, but the upshot is that 

Consider :meth:`DataFrame.fillna`. That requires checking for missing values and applying a boolean mask, selecting just the rows with no NA values. Even in NumPy, boolean indexing takes a copy of the data, and not a view.

In [13]:
airports_inplace = airports_raw.copy()
airports_inplace.dropna(inplace=True)
airports_inplace

,id,name,city,country,faa,icao,lat,lon,alt,tz,dst,tzone
0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10.0,U,Pacific/Port_Moresby
1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.788700,20,10.0,U,Pacific/Port_Moresby
2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10.0,U,Pacific/Port_Moresby
3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10.0,U,Pacific/Port_Moresby
4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.220050,146,10.0,U,Pacific/Port_Moresby
...,...,...,...,...,...,...,...,...,...,...,...,...
8102,9537,Mansons Landing Water Aerodrome,Mansons Landing,Canada,YMU,\N,50.066667,-124.983333,0,-8.0,A,America/Vancouver
8103,9538,Port McNeill Airport,Port McNeill,Canada,YMP,\N,50.575556,-127.028611,225,-8.0,A,America/Vancouver
8104,9539,Sullivan Bay Water Aerodrome,Sullivan Bay,Canada,YTG,\N,50.883333,-126.833333,0,-8.0,A,America/Vancouver
8105,9540,Deer Harbor Seaplane,Deer Harbor,United States,DHB,\N,48.618397,-123.005960,0,-8.0,A,America/Los_Angeles


Internally, that essentially does

```python
def dropna(self, inplace=True):
    data = self._data
    new_data = drop_missing_values(data)  # creates a copy!
    if inplace:
        self._data = data
    else:
        return DataFrame(data)
```

In other words, the actual operation is the same, regardless of whether `inplace=True` or `inplace=False`. The only difference is whether a new `DataFrame` object is returned, or whether your reference is updated inplace. For these types of methods, the only benefit of `inplace=True` is to avoid having to type the name of your object twice

```python
really_long_dataframe_name = really_long_dataframe_name.dropna()

# vs.

really_long_dataframe_name.dropna(inplace=True)
```

But we recommend using method chaining, which avoids the need to type the name of the object twice in the first place.

## Avoid `.values`

``DataFrame.values`` is a surprising complex attribute. The main goal is to get a NumPy representation of the data backing the DataFrame. This can be useful if you're doing lower-level numerical operations, or working with a library that needs an ndarray rather than a DataFrame.

In the simplest case, ``.values`` really does return a view on the data stored inside a DataFrame.

In [14]:
raw = np.random.randn(4, 3)
df = pd.DataFrame(raw, columns=['a', 'b', 'c'])
df

,a,b,c
0,1.835153,-0.197295,-0.521287
1,2.504146,1.277323,0.402182
2,-0.081891,0.113964,0.654143
3,0.215728,2.316733,1.519346


In [15]:
df.values.base is raw

True

However, whenever you're mixing mulitple dtypes (which is kind of the point of pandas), `.values` ceases to be a simple view.

In [16]:
cat = pd.Categorical(['a', 'b', 'c', 'd'])
df['d'] = cat
df

,a,b,c,d
0,1.835153,-0.197295,-0.521287,a
1,2.504146,1.277323,0.402182,b
2,-0.081891,0.113964,0.654143,c
3,0.215728,2.316733,1.519346,d


In [17]:
df.values

array([[1.8351532807356392, -0.19729517803690252, -0.5212865106961669,
        'a'],
       [2.5041462951796545, 1.2773227505190408, 0.40218246223160053, 'b'],
       [-0.08189092495941773, 0.11396394321264842, 0.654143407950991,
        'c'],
       [0.2157278768576361, 2.316732602185472, 1.5193463600745272, 'd']],
      dtype=object)

NumPy arrays have a single dtype for every element, which means we must find a common dtype for all the columns. In practice, this often means `object`-dtype (each element of the 2D array is a Python object). This conversion from native to object dtype is expensive in time and memory.

If you need a NumPy array from a DataFrame, we recommend using :meth:`DataFrame.to_numpy()`.

In [18]:
df.to_numpy()

array([[1.8351532807356392, -0.19729517803690252, -0.5212865106961669,
        'a'],
       [2.5041462951796545, 1.2773227505190408, 0.40218246223160053, 'b'],
       [-0.08189092495941773, 0.11396394321264842, 0.654143407950991,
        'c'],
       [0.2157278768576361, 2.316732602185472, 1.5193463600745272, 'd']],
      dtype=object)

This makes it clearer that the operation may be expensive (and offers control over whether or not to copy the data).

For :class:`Series` things are both simpler and more complex. We no longer have the issue with having to find a common dtype to accomodate multiple columns. However, not every 1-D array allowed in Pandas can be represented by NumPy.

The basics like floats are fine. And we get zero-copy access to the the original data.

In [19]:
df['a'].values

array([ 1.83515328,  2.5041463 , -0.08189092,  0.21572788])

In [20]:
df['a'].values.base is raw

True

But for extension types, this isn't necessarily true. We have two conflicting desires

1. Get a NumPy representation of the data
2. Get a zero-copy view on the original data

In [21]:
periods = pd.array(['2000', '2001', '2002', '2003'], dtype='Period[D]')
df['e'] = periods
df

,a,b,c,d,e
0,1.835153,-0.197295,-0.521287,a,2000-01-01
1,2.504146,1.277323,0.402182,b,2001-01-01
2,-0.081891,0.113964,0.654143,c,2002-01-01
3,0.215728,2.316733,1.519346,d,2003-01-01


In [22]:
df['d'].values

[a, b, c, d]
Categories (4, object): [a, b, c, d]

In [23]:
df['e'].values

array([Period('2000-01-01', 'D'), Period('2001-01-01', 'D'),
       Period('2002-01-01', 'D'), Period('2003-01-01', 'D')], dtype=object)

For the first purpose, getting a NumPy representation, we recommend :meth:`Series.to_numpy`.

In [24]:
df['d'].to_numpy()

array(['a', 'b', 'c', 'd'], dtype=object)

In [25]:
df['e'].to_numpy()

array([Period('2000-01-01', 'D'), Period('2001-01-01', 'D'),
       Period('2002-01-01', 'D'), Period('2003-01-01', 'D')], dtype=object)

In [26]:
df['a'].array

<PandasArray>
[  1.8351532807356392,   2.5041462951796545, -0.08189092495941773,
   0.2157278768576361]
Length: 4, dtype: float64

In [27]:
df['d'].array

[a, b, c, d]
Categories (4, object): [a, b, c, d]

In [28]:
df['e'].array

<PeriodArray>
['2000-01-01', '2001-01-01', '2002-01-01', '2003-01-01']
Length: 4, dtype: period[D]

See :ref:`dsintro.arraylike` for more.

## Follow Tidy Data Principles

As [Hadley Whickham](http://www.jstatsoft.org/v59/i10/paper) says, Tidy Data is about

> Structuring datasets to facilitate analysis

His three rules are that a dataset is tidy when

1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

We'll work with some data on the 2016 NBA season.

In [29]:
tables = pd.read_html("http://www.basketball-reference.com/leagues/NBA_2016_games.html")
games = tables[0]
games.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,"Tue, Oct 27, 2015",8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
1,"Tue, Oct 27, 2015",8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
2,"Tue, Oct 27, 2015",10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,"Wed, Oct 28, 2015",7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN
4,"Wed, Oct 28, 2015",7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN


We'll clean up the raw data a bit (using method chaining).

In [30]:
column_names = {'Date': 'date', 'Start (ET)': 'start',
                'Unamed: 2': 'box', 'Visitor/Neutral': 'away_team', 
                'PTS': 'away_points', 'Home/Neutral': 'home_team',
                'PTS.1': 'home_points', 'Unamed: 7': 'n_ot'}

games = (games.rename(columns=column_names)
    .dropna(thresh=4)
    [['date', 'away_team', 'away_points', 'home_team', 'home_points']]
    .assign(date=lambda x: pd.to_datetime(x['date'], format='%a, %b %d, %Y'))
    .set_index('date', append=True)
    .rename_axis(["game_id", "date"])
    .sort_index())
games.head()

,,away_team,away_points,home_team,home_points
game_id,date,,,,
0,2015-10-27,Detroit Pistons,106,Atlanta Hawks,94
1,2015-10-27,Cleveland Cavaliers,95,Chicago Bulls,97
2,2015-10-27,New Orleans Pelicans,95,Golden State Warriors,111
3,2015-10-28,Philadelphia 76ers,95,Boston Celtics,112
4,2015-10-28,Chicago Bulls,115,Brooklyn Nets,100


Consider the question **How many days of rest did each team get between each game?**
As currently structed, our dataset does not facilitate answering that question. A single team's games are spread across multiple columns (`away_team`, `home_team`).

To answer this question, the columns would be something like

date       | team_name
---------- | ---------------
2015-10-27 | Detroit Pistons
2015-10-27 | Atlanta Hawks
2015-10-27 | Cleveland Cavaliers
...        | ...

We acheive that with :meth:`DataFrame.melt`

In [31]:
tidy = (games.reset_index()
    .melt(id_vars=['game_id', 'date'], value_vars=['away_team', 'home_team'],
          value_name='team', var_name='home_or_away')
)
tidy.head()

,game_id,date,home_or_away,team
0,0,2015-10-27,away_team,Detroit Pistons
1,1,2015-10-27,away_team,Cleveland Cavaliers
2,2,2015-10-27,away_team,New Orleans Pelicans
3,3,2015-10-28,away_team,Philadelphia 76ers
4,4,2015-10-28,away_team,Chicago Bulls


Now answering the question is relatively straightforward. For each team (`.groupby('team')`), how many days passed between rows (`.date.diff().dt.days - 1`)

In [32]:
tidy.groupby('team')['date'].diff().dt.days - 1

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
     ... 
71    1.0
72    2.0
73    2.0
74    2.0
75    0.0
Name: date, Length: 76, dtype: float64

For more, visit the [full pandas documentation](pandas.pydata.org/pandas-docs/stable).